<a href="https://colab.research.google.com/github/AlkaidCheng/GSoC2021_QMLHEP/blob/main/notebooks/T09-PQC_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PQC layer

This tutorial demonstrates how to use the Parameterised Quantum Circuit (PQC) layer from Quple. The code is written using the [Tensorflow Keras Layers API](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer) and the [Tensorflow Quantum Layers API](https://www.tensorflow.org/quantum/api_docs/python/tfq/layers). 

## 0. Setup

In [ ]:
import sys
# install modules if inside google Colab environment
if 'google.colab' in sys.modules:
    !pip install tensorflow==2.4.1
    !pip install tensorflow-quantum
    !pip install quple==0.8.9.1

In [ ]:
import numpy as np

import tensorflow as tf
import tensorflow_quantum as tfq

from quple.utils.visualization import visualize_images
from quple import ParameterisedCircuit

from quple.interface.tfq.layers import PQC

## 1. Create a simple PQC model

The `PQC` layer requires two paramterised quantum circuits:

- A **data circuit** (or input circuit) for mapping classical data to the quantum states of the quantum circuit. 

- A **model circuit** that contains the trainable weights as parameters of the quantum circuit. 
 
These circuits are merged together during the quantum simulation. Inputs to the `PQC` layer will be parsed as the parameter values inside the data circuit. The output of the `PQC` layer is the expectation value of a certain measurement operators, or **readouts**, on designated qubits of the model circuit.

**Data circuit**

For the data circuit, let's start with a simple 2 qubit parameterised circuit with a layer of parameterised Rx rotation with parameter symbol "x"

In [ ]:
data_cq = ParameterisedCircuit(n_qubit=2, rotation_blocks=["RX"], parameter_symbol="x")
data_cq

(0, 0): ───Rx(x_0)───

(0, 1): ───Rx(x_1)───

**Model circuit**

For the model circuit, let's start with a simple 2 qubit parameterised circuit with a layer of parameterised Ry rotation with parameter symbol "$\theta$" (default)

In [ ]:
model_cq = ParameterisedCircuit(n_qubit=2, rotation_blocks=["RY"])
model_cq

(0, 0): ───Ry(θ_0)───

(0, 1): ───Ry(θ_1)───

**Readouts**

We will measure the $Z$ component of the qubit's quantum state

In [ ]:
readouts = model_cq.get_gate_operation("Z", model_cq.qubits)

Now create a simple quantum model with the `PQC` layer

In [ ]:
model = PQC(model_cq, data_cq, readouts, seed=2021)

## 2. Validation of the PQC model

In this section, we will verify that the implementation of the PQC model is consistent with expectation from physics

### Analytic Result from Quantum Mechanics

Initial qubit states:

$\phi_0 = |00\rangle = |0\rangle\otimes|0\rangle$

The rotaion operators used in our circuits are:

$\text{Rx}(\theta) = \begin{bmatrix} \cos(\theta/2) & -i\sin(\theta/2) \\ -i\sin(\theta/2) & \cos(\theta/2) \end{bmatrix}$,
$\text{Ry}(\theta) = \begin{bmatrix} \cos(\theta/2) & -\sin(\theta/2) \\ \sin(\theta/2) & \cos(\theta/2) \end{bmatrix}$

Final $k$-th qubit state:

$\phi_k = \text{Ry}(\theta_k)\text{Rx}(x_k)\phi_{0,k} = \begin{bmatrix} \cos(\theta_k/2)\cos(x_k/2)+i\sin(\theta_k/2)\sin(x_k/2) \\ \sin(\theta_k/2)\cos(x_k/2) -i\cos(\theta_k/2)\sin(x_k/2)\end{bmatrix}$

Expectation value of $Z$ for the $k$-th qubit:

$\langle \phi_k | Z | \phi_k\rangle$

$= |\cos(\theta_k/2)\cos(x_k/2)+i\sin(\theta_k/2)\sin(x_k/2)|^2 - |\sin(\theta_k/2)\cos(x_k/2) -i\cos(\theta_k/2)\sin(x_k/2)|^2$

$= \cos^2(\theta_k/2)\cos^2(x_k/2)+\sin^2(\theta_k/2)\sin^2(x_k/2) - \sin^2(\theta_k/2)\cos^2(x_k/2) - \cos^2(\theta_k/2)\sin^2(x_k/2)$

$=\cos(\theta_k)\cos(x_k)$

Intuitively, it's just the projection of the qubit which has just been rotated along the x and y axes by $x_k$ and $\theta_k$ back to the z axis

### Result from PQC

Trainable weights ($\theta$) from the model circuit:

In [ ]:
weights = model.weights[0].numpy()
print("================================================")
print("Trainable weight symbols:", model.symbols)
print("Trainable weight values:", weights)
print("================================================")

Trainable weight symbols: [θ_0, θ_1]
Trainable weight values: [0.03559315 5.7009115 ]


Now create 100 random samples of $x$:

In [ ]:
# ensure reproducible result
np.random.seed(2021)
sample_size = 100
x = np.random.normal(size=(sample_size, 2))

Check out the first sample:

In [ ]:
print("================================================")
print("data symbols:", model.input_symbols)
print("data values:", x[0])
print("================================================")

data symbols: [<x_0/pi>, <x_1/pi>]
data values: [1.48860905 0.67601087]


**Note**: The $1/\pi$ factor in data symbols is an artifact of how cirq treats flattened circuit parameter expression for rotation gates. The expression will eventually be multiplied by another $\pi$ factor to get back the original data value.

Now check the model output:

In [ ]:
output = model(x).numpy()

In [ ]:
print("Analytic result:")
print(np.cos(weights)*np.cos(x[0]))
print("PQC result:")
print(output[0])

Analytic result:
[0.08204278 0.65152974]
PQC result:
[0.08204272 0.6515305 ]


The difference between the analytic result and PQC result is mainly due to the precision of the default quantum simulator.

Now, let's check the PQC results are consistent with analytic resuls for all data samples:

In [ ]:
print("Are the results consistent?")
tolerance = 1e-5
analytic_result = np.cos(weights)*np.cos(x)
np.count_nonzero(analytic_result - output > tolerance) == 0

Are the results consistent?


True

## 3.  Evaluate Gradients

This section shows how to calculate gradients for PQC layer/model using `tf.GradientTape`.

Suppose that we define a loss function $L$ in terms of the model output $y = \text{PQC}(x)$:

$L = y^2 = (\text{PQC}(x))^2$

Analytically, the loss value is

$L = \cos^2(\theta_k)\cos^2(x_k)$

Gradient with respect to $\theta_k$:

$\frac{dL}{d\theta_k} = -2\cos(\theta_k)\sin(\theta_k)\cos^2(x_k) = -\sin(2\theta_k)\cos^2(x_k)$

In [ ]:
analytic_result = np.sum(-np.sin(2*weights)*(np.cos(x[:])**2), axis=0)
analytic_result

array([-3.63324594, 52.37198535])

Let's then calculate the gradient with the PQC model

In [ ]:
with tf.GradientTape() as tape:
    loss = model(x)*model(x)
gradient = tape.gradient(loss, model.trainable_variables)
gradient = gradient[0].numpy()
gradient

array([-3.6336653, 52.3725   ], dtype=float32)

## 4. Practical Example - Building a simple classification algorithm using PQC

We want to perform classification on a simple dataset $x = [x_0, x_1], 0 \leq x_0, x_1 \leq 1$ so that $ y = 0$ if $x_0 + x_1 < 1$ and $y = 1$ if $x_0 + x_1 \geq 1$

In [ ]:
sample_size = 10000
np.random.seed(2021)
x_train = np.random.uniform(size=(sample_size, 2))
y_train = np.where(np.sum(x_train, axis=1) < 1, 0, 1)

x_val = np.random.uniform(size=(sample_size//2, 2))
y_val = np.where(np.sum(x_val, axis=1) < 1, 0, 1)

x_test = np.random.uniform(size=(sample_size//2, 2))
y_test = np.where(np.sum(x_test, axis=1) < 1, 0, 1)

Now create a Keras sequential model with PQC layer

In [ ]:
from quple.data_encoding import FirstOrderPauliZEncoding
data_cq = FirstOrderPauliZEncoding(feature_dimension=2, copies=2)
data_cq

(0, 0): ───H───Rz(pi*x_0)───H───Rz(pi*x_0)───

(0, 1): ───H───Rz(pi*x_1)───H───Rz(pi*x_1)───

In [ ]:
model_cq = ParameterisedCircuit(n_qubit=2, copies=2, rotation_blocks=["RY", "RZ"],
                                entanglement_blocks=["CX"], final_rotation_layer=True)
model_cq

(0, 0): ───Ry(θ_0)───Rz(θ_2)───@───Ry(θ_4)───Rz(θ_6)───@───Ry(θ_8)───Rz(θ_10)───
                               │                       │
(0, 1): ───Ry(θ_1)───Rz(θ_3)───X───Ry(θ_5)───Rz(θ_7)───X───Ry(θ_9)───Rz(θ_11)───

In [ ]:
# measure last qubit
readout = model_cq.get_gate_operation("Z", model_cq.qubits[-1])

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(2,), dtype=tf.float32))
model.add(PQC(model_cq, data_cq, readout, seed=2021))
tf.random.set_seed(2021)
model.compile(optimizer="adam", loss="mse", metrics=['binary_accuracy'])

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=25, validation_data=(x_val, y_val))

Epoch 1/25
79/79 [==============================] - 6s 72ms/step - loss: 0.4109 - binary_accuracy: 0.6315 - val_loss: 0.3746 - val_binary_accuracy: 0.6490
Epoch 2/25
79/79 [==============================] - 5s 62ms/step - loss: 0.3651 - binary_accuracy: 0.6429 - val_loss: 0.3343 - val_binary_accuracy: 0.6544
Epoch 3/25
79/79 [==============================] - 5s 62ms/step - loss: 0.3202 - binary_accuracy: 0.6550 - val_loss: 0.3076 - val_binary_accuracy: 0.6562
Epoch 4/25
79/79 [==============================] - 5s 62ms/step - loss: 0.2992 - binary_accuracy: 0.6615 - val_loss: 0.2903 - val_binary_accuracy: 0.6560
Epoch 5/25
79/79 [==============================] - 5s 61ms/step - loss: 0.2869 - binary_accuracy: 0.6597 - val_loss: 0.2774 - val_binary_accuracy: 0.6582
Epoch 6/25
79/79 [==============================] - 5s 61ms/step - loss: 0.2764 - binary_accuracy: 0.6599 - val_loss: 0.2656 - val_binary_accuracy: 0.6666
Epoch 7/25
79/79 [==============================] - 5s 62ms/step - los

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
40/40 [==============================] - 1s 27ms/step - loss: 0.1273 - binary_accuracy: 0.8660
test loss, test acc: [0.12733320891857147, 0.8659999966621399]


In [ ]:
y_pred = model.predict(x_test).flatten()
pred_labels = np.where(y_pred < 0, 0, 1)

In [ ]:
pred_labels[:100]

array([0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0])

In [ ]:
y_test[:100]

array([0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0])